In [63]:
import pandas as pd
import re
import spacy
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import wordnet
from collections import defaultdict

In [64]:
# Load the English NLP model for lemmatization
nlp = spacy.load("en_core_web_sm")

# Load the spell checker
spell = SpellChecker()

# Lemmatization（归一化） & Correction

In [65]:
# Step 1: Lemmatization (Word Form Normalization)
def lemmatize_word(word):
    """Convert a word to its lemma form using spaCy."""
    doc = nlp(word.lower())  # Convert to lowercase and process with spaCy
    return doc[0].lemma_  # Return the lemmatized form of the first token

In [66]:
# Step 2: Spelling Correction
def correct_spelling(word):
    """Correct spelling mistakes using SpellChecker."""
    corrected = spell.correction(word)  # Attempt to correct the spelling
    return corrected if corrected else word  # Return corrected word if available, otherwise return original word

In [67]:
# Step 3: Text Cleaning
def clean_text(text):
    """Perform spelling correction on each word in the input text."""
    words = text.split()  # Split the text into individual words
    cleaned_words = []
    for word in words:
        corrected = correct_spelling(word)  # Perform spelling correction
        cleaned_words.append(corrected)  # Append the corrected word to the list
    return " ".join(cleaned_words)  # Reconstruct the cleaned text as a single string

In [69]:
# Read the Excel file
file_path = "/Users/jiazhengtian/Desktop/Research/Paul Ingram_25.2.21/Extracted Data/Combined_data.xlsx"  # Replace with your file path
df = pd.read_excel(file_path)

# Apply text cleaning to the "From" and "To" columns
df["From_Cleaned"] = df["From"].astype(str).apply(lambda x: clean_text(x))
df["To_Cleaned"] = df["To"].astype(str).apply(lambda x: clean_text(x))

df_cleaned = df.copy()
df_cleaned.drop(columns=["From_Cleaned", "To_Cleaned"], inplace=True)
df_cleaned.rename(columns={"From_Cleaned": "From", "To_Cleaned": "To"}, inplace=True)

# Values Clustering

In [42]:
# Extract unique values from the "From" and "To" columns
values = list(set(df["From"].astype(str)) | set(df["To"].astype(str)))  # Convert to string and deduplicate

In [70]:
def preprocess_text(text):
    """清理文本：去标点、转换小写、lemmatization"""
    text = text.lower().strip()
    text = re.sub(r"[^\w\s]", "", text)  # 去掉标点符号
    doc = nlp(text)
    return " ".join(sorted([token.lemma_ for token in doc if token.is_alpha]))  # 词形归一化 + 排序

In [71]:
def normalize_values(df, column_name):
    """对 DataFrame 某列的短语进行归一化"""
    value_map = {}
    unique_values = df[column_name].dropna().astype(str).unique()  # 取唯一值
    normalized_dict = defaultdict(list)

    # 处理所有唯一值，创建归一化映射
    for value in unique_values:
        normalized = preprocess_text(value)
        normalized_dict[normalized].append(value)  # 归类到相同的 key

    # 选择最常见的表达方式作为标准形式
    for norm, variations in normalized_dict.items():
        standard_value = sorted(variations, key=len)[0]  # 选最短的作为标准
        for v in variations:
            value_map[v] = standard_value  # 映射到标准表达

    # 替换 DataFrame 中的值
    df[column_name + "_Clustering"] = df[column_name].astype(str).apply(lambda x: value_map.get(x, x))
    return df

In [72]:
# **对 'From' 和 'To' 列进行归一化**
df_cleaned = normalize_values(df_cleaned, "From")
df_cleaned = normalize_values(df_cleaned, "To")

In [76]:
df_cleaned.to_excel("clustered.xlsx")